In [1]:
from math import log10
import matplotlib.pyplot as plt
import numpy as np

import pandas as pd
import os

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from dataset.data_loader_RGB import *
from utils.pytorch_ssim import *
from utils.loss import*

from models.FSRCNN_coord_model import Net
from facenet_pytorch import InceptionResnetV1

torch.manual_seed(1)
device = torch.device("cuda")

In [3]:
# FSRCNN parameters

batch_size = 24
epochs = 50
lr = 0.001
threads = 4
upscale_factor = 4

In [4]:
#img_path_low = '/media/angelo/DATEN/Datasets/Experiment_Masters/300W-3D-low-res-56/train'
#img_path_ref = '/media/angelo/DATEN/Datasets/Experiment_Masters/300W-3D-low-res-224/train'

img_path_low = '/media/angelo/DATEN/Datasets/CelebA/LR_56/train/'
img_path_ref = '/media/angelo/DATEN/Datasets/CelebA/HR/train/'

#img_path_low = '/home/jupyter/dataset/LR_56/train/'
#img_path_ref = '/home/jupyter/dataset/HR/train/'

train_set = DatasetSuperRes(img_path_low, img_path_ref)
training_data_loader = DataLoader(dataset=train_set, num_workers=threads, batch_size=batch_size, shuffle=True)

In [6]:
print('===> Building model')
model = Net().to(device)
model.weight_init(mean=0.0, std=0.2)

feature_extraction_model = InceptionResnetV1(pretrained='vggface2').eval()
face_loss = FaceIdentityLoss(feature_extraction_model).to(device)

criterion = nn.MSELoss()

optimizer = optim.Adam(model.parameters(), lr=lr, betas=(0.5, 0.999))
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.2)

===> Building model


In [7]:
out_path = 'results/'
out_model_path = 'checkpoints/'

if not os.path.exists(out_path):
    os.makedirs(out_path)    

if not os.path.exists(out_model_path):
    os.makedirs(out_model_path)   
    
results = {'avg_loss': [], 'psnr': [], 'ssim': []}

In [8]:
def train(epoch):
    epoch_loss = 0
    epoch_total_loss = 0
    model.train()
    for iteration, batch in enumerate(training_data_loader, 1):
        input_, target = batch[0].to(device), batch[1].to(device)
        
        optimizer.zero_grad()
        upsampled_img = model(input_)
        # MSE Loss for PSNR estimation
        mse_loss = criterion(upsampled_img, target)
        epoch_loss += mse_loss.item()
        # Face Loss
        total_loss = mse_loss + face_loss(upsampled_img, target)
        epoch_total_loss += total_loss.item()
        total_loss.backward()
        optimizer.step()

        print("===> Epoch[{}]({}/{}): Loss: {:.4f}".format(epoch, iteration, len(training_data_loader),
                                                           total_loss.item()))
    
    scheduler.step() # Decrease learning rate after 15 epochs to 10% of its value
    
    psnr_epoch = 10*log10(1/(epoch_loss / len(training_data_loader)))
    ssim_epoch = ssim(upsampled_img, target).item()
    avg_loss_batch = epoch_loss/len(training_data_loader)
    
    results['psnr'].append(psnr_epoch)
    results['ssim'].append(ssim_epoch)
    results['avg_loss'].append(avg_loss_batch)
    
    print("===> Epoch {} Complete: Avg. Loss: {:.4f} / PSNR: {:.4f} / SSIM {:.4f}".format(epoch, 
                                                                                          avg_loss_batch, 
                                                                                          psnr_epoch,
                                                                                          ssim_epoch))
    if epoch % (epochs // 10) == 0:
    
        data_frame = pd.DataFrame(
                data={'Avg. Loss': results['avg_loss'], 'PSNR': results['psnr'], 'SSIM': results['ssim']},
                index=range(1, epoch + 1))

        data_frame.to_csv(out_path + 'FSRCNN_coord_Loss_x' + str(upscale_factor) + '_train_results.csv', index_label='Epoch')
        
        checkpoint(epoch)
    
def checkpoint(epoch):
    path = out_model_path + "FSRCNN_coord_Loss_x{}_epoch_{}.pth".format(upscale_factor, epoch)
    torch.save(model, path)
    print("Checkpoint saved to {}".format(path))

In [9]:
epochs=10
#optimizer.param_groups[0]['lr'] = 0.001

for epoch in range(1, epochs+1):
    train(epoch)

===> Epoch[1](1/750): Loss: 1.2505
===> Epoch[1](2/750): Loss: 1.2024
===> Epoch[1](3/750): Loss: 1.1412
===> Epoch[1](4/750): Loss: 1.1344
===> Epoch[1](5/750): Loss: 1.1714
===> Epoch[1](6/750): Loss: 1.0784
===> Epoch[1](7/750): Loss: 1.1000
===> Epoch[1](8/750): Loss: 1.0817
===> Epoch[1](9/750): Loss: 1.0050
===> Epoch[1](10/750): Loss: 0.9702
===> Epoch[1](11/750): Loss: 1.0030
===> Epoch[1](12/750): Loss: 0.9445
===> Epoch[1](13/750): Loss: 0.9619
===> Epoch[1](14/750): Loss: 1.0068
===> Epoch[1](15/750): Loss: 1.0023
===> Epoch[1](16/750): Loss: 1.0191
===> Epoch[1](17/750): Loss: 0.9574
===> Epoch[1](18/750): Loss: 0.9687
===> Epoch[1](19/750): Loss: 1.0246
===> Epoch[1](20/750): Loss: 1.0027
===> Epoch[1](21/750): Loss: 0.9743
===> Epoch[1](22/750): Loss: 0.9658
===> Epoch[1](23/750): Loss: 1.0301
===> Epoch[1](24/750): Loss: 0.9685
===> Epoch[1](25/750): Loss: 1.0139
===> Epoch[1](26/750): Loss: 0.9782
===> Epoch[1](27/750): Loss: 0.9366
===> Epoch[1](28/750): Loss: 1.0183
=

===> Epoch[1](226/750): Loss: 0.7260
===> Epoch[1](227/750): Loss: 0.6853
===> Epoch[1](228/750): Loss: 0.7171
===> Epoch[1](229/750): Loss: 0.7000
===> Epoch[1](230/750): Loss: 0.6726
===> Epoch[1](231/750): Loss: 0.6502
===> Epoch[1](232/750): Loss: 0.7261
===> Epoch[1](233/750): Loss: 0.7256
===> Epoch[1](234/750): Loss: 0.7411
===> Epoch[1](235/750): Loss: 0.7706
===> Epoch[1](236/750): Loss: 0.6985
===> Epoch[1](237/750): Loss: 0.6676
===> Epoch[1](238/750): Loss: 0.6773
===> Epoch[1](239/750): Loss: 0.7465
===> Epoch[1](240/750): Loss: 0.6778
===> Epoch[1](241/750): Loss: 0.7024
===> Epoch[1](242/750): Loss: 0.6970
===> Epoch[1](243/750): Loss: 0.6705
===> Epoch[1](244/750): Loss: 0.7046
===> Epoch[1](245/750): Loss: 0.7190
===> Epoch[1](246/750): Loss: 0.6639
===> Epoch[1](247/750): Loss: 0.6673
===> Epoch[1](248/750): Loss: 0.5866
===> Epoch[1](249/750): Loss: 0.7080
===> Epoch[1](250/750): Loss: 0.6708
===> Epoch[1](251/750): Loss: 0.7031
===> Epoch[1](252/750): Loss: 0.6324
=

===> Epoch[1](448/750): Loss: 0.4925
===> Epoch[1](449/750): Loss: 0.4543
===> Epoch[1](450/750): Loss: 0.4912
===> Epoch[1](451/750): Loss: 0.5144
===> Epoch[1](452/750): Loss: 0.5165
===> Epoch[1](453/750): Loss: 0.5560
===> Epoch[1](454/750): Loss: 0.5046
===> Epoch[1](455/750): Loss: 0.4924
===> Epoch[1](456/750): Loss: 0.4368
===> Epoch[1](457/750): Loss: 0.4541
===> Epoch[1](458/750): Loss: 0.4449
===> Epoch[1](459/750): Loss: 0.4549
===> Epoch[1](460/750): Loss: 0.4796
===> Epoch[1](461/750): Loss: 0.5078
===> Epoch[1](462/750): Loss: 0.4859
===> Epoch[1](463/750): Loss: 0.4825
===> Epoch[1](464/750): Loss: 0.4393
===> Epoch[1](465/750): Loss: 0.5729
===> Epoch[1](466/750): Loss: 0.6482
===> Epoch[1](467/750): Loss: 0.5738
===> Epoch[1](468/750): Loss: 0.5952
===> Epoch[1](469/750): Loss: 0.5361
===> Epoch[1](470/750): Loss: 0.4699
===> Epoch[1](471/750): Loss: 0.4812
===> Epoch[1](472/750): Loss: 0.4943
===> Epoch[1](473/750): Loss: 0.4336
===> Epoch[1](474/750): Loss: 0.4811
=

===> Epoch[1](670/750): Loss: 0.3977
===> Epoch[1](671/750): Loss: 0.4319
===> Epoch[1](672/750): Loss: 0.3928
===> Epoch[1](673/750): Loss: 0.3952
===> Epoch[1](674/750): Loss: 0.3714
===> Epoch[1](675/750): Loss: 0.4094
===> Epoch[1](676/750): Loss: 0.3642
===> Epoch[1](677/750): Loss: 0.3636
===> Epoch[1](678/750): Loss: 0.3674
===> Epoch[1](679/750): Loss: 0.4174
===> Epoch[1](680/750): Loss: 0.4075
===> Epoch[1](681/750): Loss: 0.4088
===> Epoch[1](682/750): Loss: 0.4681
===> Epoch[1](683/750): Loss: 0.4282
===> Epoch[1](684/750): Loss: 0.4499
===> Epoch[1](685/750): Loss: 0.5140
===> Epoch[1](686/750): Loss: 0.4696
===> Epoch[1](687/750): Loss: 0.4555
===> Epoch[1](688/750): Loss: 0.4216
===> Epoch[1](689/750): Loss: 0.4158
===> Epoch[1](690/750): Loss: 0.4448
===> Epoch[1](691/750): Loss: 0.4188
===> Epoch[1](692/750): Loss: 0.4330
===> Epoch[1](693/750): Loss: 0.4288
===> Epoch[1](694/750): Loss: 0.3979
===> Epoch[1](695/750): Loss: 0.3999
===> Epoch[1](696/750): Loss: 0.3888
=

===> Epoch[2](141/750): Loss: 0.3070
===> Epoch[2](142/750): Loss: 0.3256
===> Epoch[2](143/750): Loss: 0.3759
===> Epoch[2](144/750): Loss: 0.3830
===> Epoch[2](145/750): Loss: 0.3592
===> Epoch[2](146/750): Loss: 0.3317
===> Epoch[2](147/750): Loss: 0.3099
===> Epoch[2](148/750): Loss: 0.3399
===> Epoch[2](149/750): Loss: 0.4108
===> Epoch[2](150/750): Loss: 0.3290
===> Epoch[2](151/750): Loss: 0.3925
===> Epoch[2](152/750): Loss: 0.3416
===> Epoch[2](153/750): Loss: 0.3921
===> Epoch[2](154/750): Loss: 0.3824
===> Epoch[2](155/750): Loss: 0.4077
===> Epoch[2](156/750): Loss: 0.3834
===> Epoch[2](157/750): Loss: 0.3611
===> Epoch[2](158/750): Loss: 0.3834
===> Epoch[2](159/750): Loss: 0.3464
===> Epoch[2](160/750): Loss: 0.3839
===> Epoch[2](161/750): Loss: 0.4414
===> Epoch[2](162/750): Loss: 0.4018
===> Epoch[2](163/750): Loss: 0.4253
===> Epoch[2](164/750): Loss: 0.3469
===> Epoch[2](165/750): Loss: 0.3503
===> Epoch[2](166/750): Loss: 0.3887
===> Epoch[2](167/750): Loss: 0.4198
=

...

===> Epoch[9](434/750): Loss: 0.2955
===> Epoch[9](435/750): Loss: 0.2829
===> Epoch[9](436/750): Loss: 0.2852
===> Epoch[9](437/750): Loss: 0.2935
===> Epoch[9](438/750): Loss: 0.2269
===> Epoch[9](439/750): Loss: 0.2474
===> Epoch[9](440/750): Loss: 0.2717
===> Epoch[9](441/750): Loss: 0.2768
===> Epoch[9](442/750): Loss: 0.2680
===> Epoch[9](443/750): Loss: 0.2809
===> Epoch[9](444/750): Loss: 0.2315
===> Epoch[9](445/750): Loss: 0.2775
===> Epoch[9](446/750): Loss: 0.2863
===> Epoch[9](447/750): Loss: 0.2998
===> Epoch[9](448/750): Loss: 0.2655
===> Epoch[9](449/750): Loss: 0.2822
===> Epoch[9](450/750): Loss: 0.2890
===> Epoch[9](451/750): Loss: 0.2420
===> Epoch[9](452/750): Loss: 0.2714
===> Epoch[9](453/750): Loss: 0.2612
===> Epoch[9](454/750): Loss: 0.2410
===> Epoch[9](455/750): Loss: 0.2534
===> Epoch[9](456/750): Loss: 0.3049
===> Epoch[9](457/750): Loss: 0.2879
===> Epoch[9](458/750): Loss: 0.2803
===> Epoch[9](459/750): Loss: 0.2456
===> Epoch[9](460/750): Loss: 0.2513
=

===> Epoch[9](656/750): Loss: 0.2817
===> Epoch[9](657/750): Loss: 0.2858
===> Epoch[9](658/750): Loss: 0.2391
===> Epoch[9](659/750): Loss: 0.2305
===> Epoch[9](660/750): Loss: 0.2935
===> Epoch[9](661/750): Loss: 0.2452
===> Epoch[9](662/750): Loss: 0.3353
===> Epoch[9](663/750): Loss: 0.2604
===> Epoch[9](664/750): Loss: 0.3282
===> Epoch[9](665/750): Loss: 0.2017
===> Epoch[9](666/750): Loss: 0.3184
===> Epoch[9](667/750): Loss: 0.2478
===> Epoch[9](668/750): Loss: 0.2430
===> Epoch[9](669/750): Loss: 0.3282
===> Epoch[9](670/750): Loss: 0.2687
===> Epoch[9](671/750): Loss: 0.2457
===> Epoch[9](672/750): Loss: 0.3174
===> Epoch[9](673/750): Loss: 0.2356
===> Epoch[9](674/750): Loss: 0.3373
===> Epoch[9](675/750): Loss: 0.2622
===> Epoch[9](676/750): Loss: 0.2409
===> Epoch[9](677/750): Loss: 0.2561
===> Epoch[9](678/750): Loss: 0.2534
===> Epoch[9](679/750): Loss: 0.2978
===> Epoch[9](680/750): Loss: 0.3440
===> Epoch[9](681/750): Loss: 0.2981
===> Epoch[9](682/750): Loss: 0.2822
=

===> Epoch[10](124/750): Loss: 0.2882
===> Epoch[10](125/750): Loss: 0.2778
===> Epoch[10](126/750): Loss: 0.2720
===> Epoch[10](127/750): Loss: 0.2799
===> Epoch[10](128/750): Loss: 0.2146
===> Epoch[10](129/750): Loss: 0.3188
===> Epoch[10](130/750): Loss: 0.2809
===> Epoch[10](131/750): Loss: 0.2950
===> Epoch[10](132/750): Loss: 0.2527
===> Epoch[10](133/750): Loss: 0.3232
===> Epoch[10](134/750): Loss: 0.2660
===> Epoch[10](135/750): Loss: 0.3048
===> Epoch[10](136/750): Loss: 0.2937
===> Epoch[10](137/750): Loss: 0.2822
===> Epoch[10](138/750): Loss: 0.2854
===> Epoch[10](139/750): Loss: 0.2508
===> Epoch[10](140/750): Loss: 0.2922
===> Epoch[10](141/750): Loss: 0.2449
===> Epoch[10](142/750): Loss: 0.2431
===> Epoch[10](143/750): Loss: 0.2879
===> Epoch[10](144/750): Loss: 0.2856
===> Epoch[10](145/750): Loss: 0.2806
===> Epoch[10](146/750): Loss: 0.2628
===> Epoch[10](147/750): Loss: 0.2560
===> Epoch[10](148/750): Loss: 0.2662
===> Epoch[10](149/750): Loss: 0.3228
===> Epoch[1

===> Epoch[10](340/750): Loss: 0.3016
===> Epoch[10](341/750): Loss: 0.2330
===> Epoch[10](342/750): Loss: 0.2727
===> Epoch[10](343/750): Loss: 0.2683
===> Epoch[10](344/750): Loss: 0.2771
===> Epoch[10](345/750): Loss: 0.2461
===> Epoch[10](346/750): Loss: 0.2586
===> Epoch[10](347/750): Loss: 0.2318
===> Epoch[10](348/750): Loss: 0.2527
===> Epoch[10](349/750): Loss: 0.3300
===> Epoch[10](350/750): Loss: 0.2508
===> Epoch[10](351/750): Loss: 0.2774
===> Epoch[10](352/750): Loss: 0.2710
===> Epoch[10](353/750): Loss: 0.2190
===> Epoch[10](354/750): Loss: 0.2840
===> Epoch[10](355/750): Loss: 0.2826
===> Epoch[10](356/750): Loss: 0.2638
===> Epoch[10](357/750): Loss: 0.3198
===> Epoch[10](358/750): Loss: 0.2666
===> Epoch[10](359/750): Loss: 0.2560
===> Epoch[10](360/750): Loss: 0.3030
===> Epoch[10](361/750): Loss: 0.2707
===> Epoch[10](362/750): Loss: 0.2566
===> Epoch[10](363/750): Loss: 0.3094
===> Epoch[10](364/750): Loss: 0.2022
===> Epoch[10](365/750): Loss: 0.2403
===> Epoch[1

===> Epoch[10](556/750): Loss: 0.2972
===> Epoch[10](557/750): Loss: 0.2439
===> Epoch[10](558/750): Loss: 0.2928
===> Epoch[10](559/750): Loss: 0.2774
===> Epoch[10](560/750): Loss: 0.2702
===> Epoch[10](561/750): Loss: 0.2736
===> Epoch[10](562/750): Loss: 0.2741
===> Epoch[10](563/750): Loss: 0.2307
===> Epoch[10](564/750): Loss: 0.3102
===> Epoch[10](565/750): Loss: 0.2276
===> Epoch[10](566/750): Loss: 0.2537
===> Epoch[10](567/750): Loss: 0.2759
===> Epoch[10](568/750): Loss: 0.2702
===> Epoch[10](569/750): Loss: 0.2959
===> Epoch[10](570/750): Loss: 0.2851
===> Epoch[10](571/750): Loss: 0.2974
===> Epoch[10](572/750): Loss: 0.2401
===> Epoch[10](573/750): Loss: 0.2809
===> Epoch[10](574/750): Loss: 0.2823
===> Epoch[10](575/750): Loss: 0.2767
===> Epoch[10](576/750): Loss: 0.2833
===> Epoch[10](577/750): Loss: 0.3143
===> Epoch[10](578/750): Loss: 0.3163
===> Epoch[10](579/750): Loss: 0.3121
===> Epoch[10](580/750): Loss: 0.2540
===> Epoch[10](581/750): Loss: 0.2639
===> Epoch[1

In [11]:
for epoch in range(21, 31):
    train(epoch)

===> Epoch[21](1/750): Loss: 0.2873
===> Epoch[21](2/750): Loss: 0.2589
===> Epoch[21](3/750): Loss: 0.2466
===> Epoch[21](4/750): Loss: 0.2654
===> Epoch[21](5/750): Loss: 0.2715
===> Epoch[21](6/750): Loss: 0.3008
===> Epoch[21](7/750): Loss: 0.2786
===> Epoch[21](8/750): Loss: 0.2542
===> Epoch[21](9/750): Loss: 0.2752
===> Epoch[21](10/750): Loss: 0.3007
===> Epoch[21](11/750): Loss: 0.3237
===> Epoch[21](12/750): Loss: 0.2755
===> Epoch[21](13/750): Loss: 0.2980
===> Epoch[21](14/750): Loss: 0.2424
===> Epoch[21](15/750): Loss: 0.2870
===> Epoch[21](16/750): Loss: 0.2154
===> Epoch[21](17/750): Loss: 0.2852
===> Epoch[21](18/750): Loss: 0.2742
===> Epoch[21](19/750): Loss: 0.2460
===> Epoch[21](20/750): Loss: 0.2511
===> Epoch[21](21/750): Loss: 0.2522
===> Epoch[21](22/750): Loss: 0.2571
===> Epoch[21](23/750): Loss: 0.2385
===> Epoch[21](24/750): Loss: 0.2351
===> Epoch[21](25/750): Loss: 0.2478
===> Epoch[21](26/750): Loss: 0.2637
===> Epoch[21](27/750): Loss: 0.2125
===> Epoch

===> Epoch[21](220/750): Loss: 0.2476
===> Epoch[21](221/750): Loss: 0.1978
===> Epoch[21](222/750): Loss: 0.2738
===> Epoch[21](223/750): Loss: 0.2657
===> Epoch[21](224/750): Loss: 0.2427
===> Epoch[21](225/750): Loss: 0.2526
===> Epoch[21](226/750): Loss: 0.2684
===> Epoch[21](227/750): Loss: 0.2620
===> Epoch[21](228/750): Loss: 0.2741
===> Epoch[21](229/750): Loss: 0.2479
===> Epoch[21](230/750): Loss: 0.2511
===> Epoch[21](231/750): Loss: 0.2415
===> Epoch[21](232/750): Loss: 0.2570
===> Epoch[21](233/750): Loss: 0.2902
===> Epoch[21](234/750): Loss: 0.2345
===> Epoch[21](235/750): Loss: 0.2700
===> Epoch[21](236/750): Loss: 0.2555
===> Epoch[21](237/750): Loss: 0.2318
===> Epoch[21](238/750): Loss: 0.2749
===> Epoch[21](239/750): Loss: 0.2189
===> Epoch[21](240/750): Loss: 0.2568
===> Epoch[21](241/750): Loss: 0.3024
===> Epoch[21](242/750): Loss: 0.2744
===> Epoch[21](243/750): Loss: 0.3041
===> Epoch[21](244/750): Loss: 0.2424
===> Epoch[21](245/750): Loss: 0.2224
===> Epoch[2

===> Epoch[21](436/750): Loss: 0.2457
===> Epoch[21](437/750): Loss: 0.2630
===> Epoch[21](438/750): Loss: 0.2304
===> Epoch[21](439/750): Loss: 0.3014
===> Epoch[21](440/750): Loss: 0.2396
===> Epoch[21](441/750): Loss: 0.2799
===> Epoch[21](442/750): Loss: 0.2726
===> Epoch[21](443/750): Loss: 0.2679
===> Epoch[21](444/750): Loss: 0.3075
===> Epoch[21](445/750): Loss: 0.2618
===> Epoch[21](446/750): Loss: 0.2105
===> Epoch[21](447/750): Loss: 0.2795
===> Epoch[21](448/750): Loss: 0.2204
===> Epoch[21](449/750): Loss: 0.2386
===> Epoch[21](450/750): Loss: 0.3002
===> Epoch[21](451/750): Loss: 0.2577
===> Epoch[21](452/750): Loss: 0.2855
===> Epoch[21](453/750): Loss: 0.2501
===> Epoch[21](454/750): Loss: 0.2627
===> Epoch[21](455/750): Loss: 0.2411
===> Epoch[21](456/750): Loss: 0.2822
===> Epoch[21](457/750): Loss: 0.2324
===> Epoch[21](458/750): Loss: 0.2383
===> Epoch[21](459/750): Loss: 0.2613
===> Epoch[21](460/750): Loss: 0.2762
===> Epoch[21](461/750): Loss: 0.2318
===> Epoch[2

===> Epoch[21](652/750): Loss: 0.2563
===> Epoch[21](653/750): Loss: 0.2831
===> Epoch[21](654/750): Loss: 0.2793
===> Epoch[21](655/750): Loss: 0.2746
===> Epoch[21](656/750): Loss: 0.2723
===> Epoch[21](657/750): Loss: 0.2586
===> Epoch[21](658/750): Loss: 0.2508
===> Epoch[21](659/750): Loss: 0.2850
===> Epoch[21](660/750): Loss: 0.2420
===> Epoch[21](661/750): Loss: 0.2632
===> Epoch[21](662/750): Loss: 0.3060
===> Epoch[21](663/750): Loss: 0.2333
===> Epoch[21](664/750): Loss: 0.2849
===> Epoch[21](665/750): Loss: 0.2914
===> Epoch[21](666/750): Loss: 0.2738
===> Epoch[21](667/750): Loss: 0.3079
===> Epoch[21](668/750): Loss: 0.2106
===> Epoch[21](669/750): Loss: 0.2868
===> Epoch[21](670/750): Loss: 0.2169
===> Epoch[21](671/750): Loss: 0.2955
===> Epoch[21](672/750): Loss: 0.2401
===> Epoch[21](673/750): Loss: 0.2574
===> Epoch[21](674/750): Loss: 0.2421
===> Epoch[21](675/750): Loss: 0.2300
===> Epoch[21](676/750): Loss: 0.2545
===> Epoch[21](677/750): Loss: 0.2667
===> Epoch[2

===> Epoch[22](117/750): Loss: 0.2267
===> Epoch[22](118/750): Loss: 0.2874
===> Epoch[22](119/750): Loss: 0.2403
===> Epoch[22](120/750): Loss: 0.2208
===> Epoch[22](121/750): Loss: 0.2503
===> Epoch[22](122/750): Loss: 0.2454
===> Epoch[22](123/750): Loss: 0.2541
===> Epoch[22](124/750): Loss: 0.2276
===> Epoch[22](125/750): Loss: 0.2813
===> Epoch[22](126/750): Loss: 0.2255
===> Epoch[22](127/750): Loss: 0.3165
===> Epoch[22](128/750): Loss: 0.2509
===> Epoch[22](129/750): Loss: 0.2278
===> Epoch[22](130/750): Loss: 0.2156
===> Epoch[22](131/750): Loss: 0.2273
===> Epoch[22](132/750): Loss: 0.1971
===> Epoch[22](133/750): Loss: 0.2831
===> Epoch[22](134/750): Loss: 0.2260
===> Epoch[22](135/750): Loss: 0.2695
===> Epoch[22](136/750): Loss: 0.2372
===> Epoch[22](137/750): Loss: 0.2293
===> Epoch[22](138/750): Loss: 0.2612
===> Epoch[22](139/750): Loss: 0.2847
===> Epoch[22](140/750): Loss: 0.2798
===> Epoch[22](141/750): Loss: 0.2368
===> Epoch[22](142/750): Loss: 0.2639
===> Epoch[2

...

===> Epoch[29](478/750): Loss: 0.2164
===> Epoch[29](479/750): Loss: 0.2436
===> Epoch[29](480/750): Loss: 0.2498
===> Epoch[29](481/750): Loss: 0.2689
===> Epoch[29](482/750): Loss: 0.2563
===> Epoch[29](483/750): Loss: 0.2182
===> Epoch[29](484/750): Loss: 0.2571
===> Epoch[29](485/750): Loss: 0.2355
===> Epoch[29](486/750): Loss: 0.2834
===> Epoch[29](487/750): Loss: 0.2425
===> Epoch[29](488/750): Loss: 0.2462
===> Epoch[29](489/750): Loss: 0.2926
===> Epoch[29](490/750): Loss: 0.2861
===> Epoch[29](491/750): Loss: 0.2464
===> Epoch[29](492/750): Loss: 0.2971
===> Epoch[29](493/750): Loss: 0.2176
===> Epoch[29](494/750): Loss: 0.2549
===> Epoch[29](495/750): Loss: 0.2522
===> Epoch[29](496/750): Loss: 0.2556
===> Epoch[29](497/750): Loss: 0.2600
===> Epoch[29](498/750): Loss: 0.2183
===> Epoch[29](499/750): Loss: 0.2227
===> Epoch[29](500/750): Loss: 0.2305
===> Epoch[29](501/750): Loss: 0.2196
===> Epoch[29](502/750): Loss: 0.2494
===> Epoch[29](503/750): Loss: 0.2834
===> Epoch[2

===> Epoch[29](694/750): Loss: 0.2819
===> Epoch[29](695/750): Loss: 0.2152
===> Epoch[29](696/750): Loss: 0.2763
===> Epoch[29](697/750): Loss: 0.2287
===> Epoch[29](698/750): Loss: 0.2337
===> Epoch[29](699/750): Loss: 0.2734
===> Epoch[29](700/750): Loss: 0.2566
===> Epoch[29](701/750): Loss: 0.2337
===> Epoch[29](702/750): Loss: 0.2432
===> Epoch[29](703/750): Loss: 0.2823
===> Epoch[29](704/750): Loss: 0.2939
===> Epoch[29](705/750): Loss: 0.2172
===> Epoch[29](706/750): Loss: 0.2394
===> Epoch[29](707/750): Loss: 0.1795
===> Epoch[29](708/750): Loss: 0.2608
===> Epoch[29](709/750): Loss: 0.2375
===> Epoch[29](710/750): Loss: 0.2341
===> Epoch[29](711/750): Loss: 0.2675
===> Epoch[29](712/750): Loss: 0.2132
===> Epoch[29](713/750): Loss: 0.3108
===> Epoch[29](714/750): Loss: 0.3125
===> Epoch[29](715/750): Loss: 0.2098
===> Epoch[29](716/750): Loss: 0.1816
===> Epoch[29](717/750): Loss: 0.2339
===> Epoch[29](718/750): Loss: 0.2976
===> Epoch[29](719/750): Loss: 0.2795
===> Epoch[2

===> Epoch[30](159/750): Loss: 0.2743
===> Epoch[30](160/750): Loss: 0.2528
===> Epoch[30](161/750): Loss: 0.2258
===> Epoch[30](162/750): Loss: 0.2733
===> Epoch[30](163/750): Loss: 0.2444
===> Epoch[30](164/750): Loss: 0.2291
===> Epoch[30](165/750): Loss: 0.2675
===> Epoch[30](166/750): Loss: 0.2858
===> Epoch[30](167/750): Loss: 0.2460
===> Epoch[30](168/750): Loss: 0.2305
===> Epoch[30](169/750): Loss: 0.2644
===> Epoch[30](170/750): Loss: 0.2184
===> Epoch[30](171/750): Loss: 0.2296
===> Epoch[30](172/750): Loss: 0.2371
===> Epoch[30](173/750): Loss: 0.2567
===> Epoch[30](174/750): Loss: 0.2730
===> Epoch[30](175/750): Loss: 0.2609
===> Epoch[30](176/750): Loss: 0.2229
===> Epoch[30](177/750): Loss: 0.2485
===> Epoch[30](178/750): Loss: 0.2969
===> Epoch[30](179/750): Loss: 0.2447
===> Epoch[30](180/750): Loss: 0.2408
===> Epoch[30](181/750): Loss: 0.2779
===> Epoch[30](182/750): Loss: 0.2436
===> Epoch[30](183/750): Loss: 0.2413
===> Epoch[30](184/750): Loss: 0.2412
===> Epoch[3

===> Epoch[30](375/750): Loss: 0.2497
===> Epoch[30](376/750): Loss: 0.1937
===> Epoch[30](377/750): Loss: 0.2193
===> Epoch[30](378/750): Loss: 0.2437
===> Epoch[30](379/750): Loss: 0.2543
===> Epoch[30](380/750): Loss: 0.2731
===> Epoch[30](381/750): Loss: 0.2351
===> Epoch[30](382/750): Loss: 0.2262
===> Epoch[30](383/750): Loss: 0.2634
===> Epoch[30](384/750): Loss: 0.2469
===> Epoch[30](385/750): Loss: 0.2356
===> Epoch[30](386/750): Loss: 0.2392
===> Epoch[30](387/750): Loss: 0.2644
===> Epoch[30](388/750): Loss: 0.2637
===> Epoch[30](389/750): Loss: 0.3322
===> Epoch[30](390/750): Loss: 0.2976
===> Epoch[30](391/750): Loss: 0.2729
===> Epoch[30](392/750): Loss: 0.2181
===> Epoch[30](393/750): Loss: 0.2493
===> Epoch[30](394/750): Loss: 0.2201
===> Epoch[30](395/750): Loss: 0.2331
===> Epoch[30](396/750): Loss: 0.2377
===> Epoch[30](397/750): Loss: 0.2594
===> Epoch[30](398/750): Loss: 0.2877
===> Epoch[30](399/750): Loss: 0.2502
===> Epoch[30](400/750): Loss: 0.3022
===> Epoch[3

===> Epoch[30](591/750): Loss: 0.2822
===> Epoch[30](592/750): Loss: 0.2073
===> Epoch[30](593/750): Loss: 0.2300
===> Epoch[30](594/750): Loss: 0.2172
===> Epoch[30](595/750): Loss: 0.2615
===> Epoch[30](596/750): Loss: 0.2206
===> Epoch[30](597/750): Loss: 0.2159
===> Epoch[30](598/750): Loss: 0.2717
===> Epoch[30](599/750): Loss: 0.2709
===> Epoch[30](600/750): Loss: 0.2611
===> Epoch[30](601/750): Loss: 0.2498
===> Epoch[30](602/750): Loss: 0.2070
===> Epoch[30](603/750): Loss: 0.2673
===> Epoch[30](604/750): Loss: 0.2663
===> Epoch[30](605/750): Loss: 0.2577
===> Epoch[30](606/750): Loss: 0.2702
===> Epoch[30](607/750): Loss: 0.2791
===> Epoch[30](608/750): Loss: 0.2115
===> Epoch[30](609/750): Loss: 0.2712
===> Epoch[30](610/750): Loss: 0.2804
===> Epoch[30](611/750): Loss: 0.2010
===> Epoch[30](612/750): Loss: 0.2816
===> Epoch[30](613/750): Loss: 0.2840
===> Epoch[30](614/750): Loss: 0.2581
===> Epoch[30](615/750): Loss: 0.2256
===> Epoch[30](616/750): Loss: 0.2681
===> Epoch[3